In [1]:
import pandas as pd
import numpy as np
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import gc
import h5py

from PIL import Image
from pyts.image import RecurrencePlot, GramianAngularField
from pyts.datasets import load_gunpoint
from pyts.image import MarkovTransitionField
from pyts.image import GramianAngularField

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import tensorflow.python.keras.backend as K

from keras import optimizers
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten, Conv3D,Conv2D, MaxPooling3D, MaxPooling2D, Dropout, LSTM
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from keras.applications import VGG16
from keras.optimizers import Adam

Using TensorFlow backend.


# Make Graph

In [ ]:
check = 1

save_path = 'E:/Data/ERP_array/graph/'
data = pd.read_csv('./input/ERPdata.csv', header=0)
#data.drop('time_ms', axis=1, inplace=True)

for i in np.unique(data.subject):
    start = time.time()
    
    for j in np.unique(data[data.subject==i].condition):
        condition_data = data[(data.subject==i) & (data.condition==j)]
        for k in range(2,11):
            plt.plot(condition_data.iloc[:,k])
            #plt.axis('off')       
            plt.savefig(save_path+'sub%d_con%d_%d.jpg' %(i,j,(k-1)), dpi=100)
            plt.cla()
    
    print("%d time : "%check, time.time() - start)
    check  = check+1

In [ ]:
# make vgg data 1~9
path = 'E:/Data/ERP_array/graph/'
save_path = 'E:/Data/ERP_array/vgg_data_graph/'
files = os.listdir(path)

for i in range(1,10):
    file_list = []
    
    for file in files:
        if (file[4] == '_') and 'sub'+str(i) in file:
            file_list.append(file)
    
    for file in file_list:
        if not os.path.isdir(save_path+'/%d'%i):
            os.mkdir(save_path+'/%d'%i)
        
        
        img = mpimg.imread(path+file)
        img = Image.fromarray(img)
        img = img.resize((224,224))
        img_arr = np.asarray(img)
        input_data = img_arr / 255

        np.savez_compressed(save_path+str(i)+'/'+file, input_data)

In [ ]:
# make vgg data 10~81
path = 'E:/Data/ERP_array/graph/'
save_path = 'E:/Data/ERP_array/vgg_data_graph/'
files = os.listdir(path)

for i in range(10,82):
    file_list = []
    
    for file in files:
        if 'sub'+str(i) in file:
            file_list.append(file)
    
    for file in file_list:
        if not os.path.isdir(save_path+'/%d'%i):
            os.mkdir(save_path+'/%d'%i)
       
        img = mpimg.imread(path+file)
        img = Image.fromarray(img)
        img = img.resize((224,224))
        img_arr = np.asarray(img)
        input_data = img_arr / 255

        np.savez_compressed(save_path+str(i)+'/'+file, input_data)

In [2]:
#Learning Curve

def draw_plot(hist):
    fig, loss_ax = plt.subplots()

    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

    acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')

    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    acc_ax.set_ylabel('accuray')

    loss_ax.legend(loc='upper left')
    acc_ax.legend(loc='lower left')

    plt.show()

# Model

In [3]:
def create_model():
    pre_trained_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    pre_trained_vgg.trainable = False

    model = Sequential()
    model.add(pre_trained_vgg)
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    #model.add(Dropout(rate=0.2))
    model.add(Dense(2048, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.5))
    #model.add(Dense(256, activation='relu'))
    #model.add(Dense(64, activation='relu'))
    #model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
    
    #return model
    return model, model.get_weights()


# Make Input Data

In [4]:
label_path = './input/demographic.csv'
label_data = pd.read_csv(label_path)
label_list = label_data['group']

In [5]:
path = 'E:/Data/ERP_array/vgg_data_graph/'
x_data = []
y_data = []

for i in range(1,82):
    file_list = os.listdir(path+'%d' %i)
    #del_con(file_list)
    label = label_list[i-1]
    
    label = np.repeat(label, len(file_list))
    #for k in range(30):
    y_data = np.append(y_data, label)
    
    for file in file_list:      
        data = np.load(path+str(i)+'/'+file)
        data = data['arr_0']
        x_data.append(data)
        
x_data = np.array(x_data)
y_data = np.array(y_data)
#x_data = x_data.reshape(-1,224,224, 1)

In [6]:
x_data.shape

(2187, 224, 224, 3)

In [8]:
kfold = KFold(n_splits=10, shuffle=True, random_state=1004)

In [9]:
path = 'E:/Data/ERP_array/vgg_data/'

In [10]:
def init_weight(same_old_model, first_weights):
    same_old_model.set_weights(weights)

In [ ]:
def cf_mat(test):
    y_pred = model.predict_classes(x_data[test])
    return confusion_matrix(y_data[test], y_pred)

In [11]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=50, restore_best_weights=True)

# Train & Test

In [12]:
num = 0
k_accuracy = list()
accuracy = list()

model, weights = create_model()
for train, test in kfold.split(x_data):
    #cuda.select_device (0)
    #model = create_model()
    num = num+1
    print("---------------------------epoch : ",num,"---------------------------")
    init_weight(model, weights)
    model.fit(x_data[train], y_data[train], epochs=200, batch_size=20, callbacks=[es])
    accuracy =  (model.evaluate(x_data[test], y_data[test]))
    k_accuracy.append(accuracy)
    
    mat.append(cf_mat(test))

---------------------------epoch :  1 ---------------------------
Epoch 1/200
1968/1968 [==============================] - 14s 7ms/step - loss: 0.7213 - accuracy: 0.5925
Epoch 2/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.6655 - accuracy: 0.6250
Epoch 3/200
1968/1968 [==============================] - 11s 5ms/step - loss: 0.5873 - accuracy: 0.7007
Epoch 4/200
1968/1968 [==============================] - 11s 5ms/step - loss: 0.5359 - accuracy: 0.7302
Epoch 5/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.4616 - accuracy: 0.7749
Epoch 6/200
1968/1968 [==============================] - 11s 5ms/step - loss: 0.4375 - accuracy: 0.7998
Epoch 7/200
1968/1968 [==============================] - 11s 5ms/step - loss: 0.4264 - accuracy: 0.8069
Epoch 8/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.4109 - accuracy: 0.8003
Epoch 9/200
1968/1968 [==============================] - 11s 5ms/step - loss: 0.3633 - accuracy: 0.829

1968/1968 [==============================] - 11s 6ms/step - loss: 0.1662 - accuracy: 0.9233
Epoch 72/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1397 - accuracy: 0.9370
Epoch 73/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1178 - accuracy: 0.9482
Epoch 74/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1531 - accuracy: 0.9294
Epoch 75/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1668 - accuracy: 0.9212
Epoch 76/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1434 - accuracy: 0.9294
Epoch 77/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1550 - accuracy: 0.9212
Epoch 78/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1896 - accuracy: 0.9050
Epoch 79/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1757 - accuracy: 0.9197
Epoch 80/200
1968/1968 [==============================] - 11s 6ms/st

1968/1968 [==============================] - 11s 6ms/step - loss: 0.1046 - accuracy: 0.9593
Epoch 50/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1437 - accuracy: 0.9395
Epoch 51/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1170 - accuracy: 0.9482
Epoch 52/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1150 - accuracy: 0.9522
Epoch 53/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1158 - accuracy: 0.9512
Epoch 54/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.0983 - accuracy: 0.9619
Epoch 55/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1131 - accuracy: 0.9563
Epoch 56/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1283 - accuracy: 0.9472
Epoch 57/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1260 - accuracy: 0.9482
Epoch 58/200
1968/1968 [==============================] - 11s 6ms/st

1968/1968 [==============================] - 11s 6ms/step - loss: 0.2102 - accuracy: 0.9055
Epoch 19/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1776 - accuracy: 0.9314
Epoch 20/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1647 - accuracy: 0.9309
Epoch 21/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1738 - accuracy: 0.9284
Epoch 22/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1483 - accuracy: 0.9400
Epoch 23/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1675 - accuracy: 0.9263
Epoch 24/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1332 - accuracy: 0.9461
Epoch 25/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1476 - accuracy: 0.9451
Epoch 26/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1355 - accuracy: 0.9492
Epoch 27/200
1968/1968 [==============================] - 11s 6ms/st

1968/1968 [==============================] - 11s 6ms/step - loss: 0.0964 - accuracy: 0.9629
Epoch 52/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.0994 - accuracy: 0.9629
Epoch 53/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1205 - accuracy: 0.9436
Epoch 54/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1113 - accuracy: 0.9548
Epoch 55/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.0916 - accuracy: 0.9649
Epoch 56/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1432 - accuracy: 0.9431
Epoch 57/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1322 - accuracy: 0.9461
Epoch 58/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1808 - accuracy: 0.9238
Epoch 59/200
1968/1968 [==============================] - 11s 6ms/step - loss: 0.1826 - accuracy: 0.9233
Epoch 60/200
1968/1968 [==============================] - 11s 6ms/st

1969/1969 [==============================] - 11s 6ms/step - loss: 0.1418 - accuracy: 0.9314
Epoch 114/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1117 - accuracy: 0.9446
Epoch 115/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1357 - accuracy: 0.9391
Epoch 116/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1151 - accuracy: 0.9441
Epoch 117/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1230 - accuracy: 0.9309
Restoring model weights from the end of the best epoch
Epoch 00117: early stopping
218/218 [==============================] - 2s 9ms/step
---------------------------epoch :  9 ---------------------------
Epoch 1/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.8639 - accuracy: 0.5759
Epoch 2/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.6307 - accuracy: 0.6597
Epoch 3/200
1969/1969 [==============================] - 11s 6ms/step - 

1969/1969 [==============================] - 11s 6ms/step - loss: 0.1133 - accuracy: 0.9467
Epoch 59/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1887 - accuracy: 0.9132
Epoch 60/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1252 - accuracy: 0.9487
Epoch 61/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1287 - accuracy: 0.9401
Epoch 62/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1695 - accuracy: 0.9253
Epoch 63/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1073 - accuracy: 0.9502
Epoch 64/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1016 - accuracy: 0.9573
Epoch 65/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1437 - accuracy: 0.9421
Epoch 66/200
1969/1969 [==============================] - 11s 6ms/step - loss: 0.1428 - accuracy: 0.9416
Epoch 67/200
1969/1969 [==============================] - 11s 6ms/st

In [13]:
k_accuracy

[[0.16611912969040543, 0.9269406199455261],
 [0.19720072035579922, 0.9269406199455261],
 [0.2941980438772589, 0.9178082346916199],
 [0.25754983462903597, 0.8995434045791626],
 [0.33832256976710673, 0.8812785148620605],
 [0.5818634090921735, 0.8401826620101929],
 [0.2802430896182038, 0.8858447670936584],
 [0.3366224760309272, 0.9036697149276733],
 [0.11320941384300726, 0.9449541568756104],
 [0.26008532134764784, 0.89449542760849]]

In [ ]:
mat